In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
from analysis.lib.fitting import fit, common
from analysis.lib.pq import pq_tools;reload(pq_tools)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

thetas = ['pi/6']

In [ ]:
vals = []

for theta in thetas:
    
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

    Density_Matrix = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Density_Matrix.get_tstamps_and_offsets(contains = 'Purify',verbose = False)


#     ### load the data
    Density_Matrix.load_raw_data()
    Density_Matrix.correct_pq_times()

In [ ]:
### filter parameters
max_reps_w2 = 50

In [ ]:
### apply filters to data
Density_Matrix.apply_temporal_filters_to_prefiltered_data()
Density_Matrix.apply_sync_filter_w1_w2(verbose = True,max_w2 = max_reps_w2)
Density_Matrix.apply_is_purified_filter(signature = '11', verbose = True) 
Density_Matrix.apply_CR_before_filter(verbose=False)
Density_Matrix.apply_CR_after_filter(verbose=False)
Density_Matrix.attach_state_filtered_syncs(verbose = False)

In [ ]:
### acquire density matrix
Density_Matrix.get_DM_correlations(verbose = True,apply_ROC = True)
Density_Matrix.reconstruct_DMs(verbose = True,max_likelihood = False)
dm_p,dm_p_u,dm_m,dm_m_u=Density_Matrix.reconstruct_DMs(verbose = False,max_likelihood = False)

In [ ]:
reload(pa)
print dm_p
pa.plot_3D_bars(dm_p,dm_u_re = dm_p_u.real,dm_u_im = dm_p_u.imag,name='same_detector')
pa.plot_3D_bars(dm_m,dm_u_re = dm_m_u.real,dm_u_im = dm_m_u.imag,name='different_detector')

In [ ]:
print 'reconstructed DMs'
print np.round(dm_p,2)
print
print np.round(dm_m,2)
print 'The statistical uncertainties'
print np.round(dm_p_u,2)
print
print np.round(dm_m_u,2)